# Using `libfec` to (almost) replicate an LA Times campaign finance story

Gabrielle LaMarr LeMee and Sandhya Kambhampati of the LA Times published 
[*"More than half of Harris donors had not given to Biden’s 2024 campaign"*](https://www.latimes.com/politics/story/2024-08-27/where-kamala-harris-most-new-donors-july-2024-election)
on August 27th of 2024. It details the composition of Kamala Harris supporters in the weeks following Biden's resignation
and subsequent endorsement of Kamala Harris' presidential campaign. 

Let's try to recreate the story, using the `libfec` CLI and SQLite!

## Small caveat: not a full replication

The LA Times article uses a few different committee filings to source their data, while here we will only focus on receipts reported 
by ActBlue. This is nearly everything, but the data found here will be slightly incomplete compared to the LA Times analysis. 

## Step 1: Sourcing all ActBlue data

The LAT article contains this nerdbox about the dataset they analyzed:

> *The Times analysis is based on Federal Election Commission filings as of Aug. 20. The Times analysis includes itemized contributions to **Harris for President, Harris Victory Fund, Harris Baldwin Victory Fund, and Harris Action Fund**. Some contributions to these committees were **given through ActBlue**. A donor was determined to be a prior Biden donor if a donation from their unique combination of first name, last name and ZIP Code had been made **from Jan. 2023 to July 20, 2024**.*

So we're looking at data as early as January 2023, from ActBlue (for this replication), and for those four specific committees. 

We can use `libfec` to download this data directly from the ActBlue filings, we'll just need to find the FEC IDs. Looking at [the ActBlue FEC commitee webpage](https://docquery.fec.gov/cgi-bin/forms/C00401224/), we can see all the filings that ActBlue have submitted, and we care about the `F3X` forms. This is [FEC Form 3X](https://www.fec.gov/resources/cms-content/documents/policy-guidance/fecfrm3x.pdf), aka "Report of Receipts and Disbursements", that committees like ActBlue file. ActBlue files these every month (`MAY MONLTHY)

In [20]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
%sql sqlite:///tmp.db

In [22]:
%%sql
select * from biden_harris_itemizations limit 10;

Running query in 'sqlite:///tmp.db'

filing_id,form_type,filer_committee_id_number,transaction_id,back_reference_tran_id_number,back_reference_sched_name,entity_type,contributor_organization_name,contributor_last_name,contributor_first_name,contributor_middle_name,contributor_prefix,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip_code,election_code,election_other_description,contribution_date,contribution_amount,contribution_aggregate,contribution_purpose_descrip,contributor_employer,contributor_occupation,donor_committee_fec_id,donor_committee_name,donor_candidate_fec_id,donor_candidate_last_name,donor_candidate_first_name,donor_candidate_middle_name,donor_candidate_prefix,donor_candidate_suffix,donor_candidate_office,donor_candidate_state,donor_candidate_district,conduit_name,conduit_street1,conduit_street2,conduit_city,conduit_state,conduit_zip_code,memo_code,memo_text_description,reference_code,contributor_zip_code5,contributor_id
1720554,SA11AI,C00401224,SA11AI_535329455,,,IND,,A,DA BABY,,,,122 DEERHUNTER LN,,POWDER SPRINGS,GA,30127,,,2023-06-30,1.0,1.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,30127,da babya30127
1720554,SA11AI,C00401224,SA11AI_534850611,,,IND,,A,DIANA,,,,6808 ZINNIA CT,,CARLSBAD,CA,92011,,,2023-06-28,10.0,10.0,Earmark,ANTHEM,PROJECT DIRECTOR,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,92011,dianaa92011
1720554,SA11AI,C00401224,SA11AI_529522643,,,IND,,A,JOHN,,,,1636 ADRIEN DR,,CAMPBELL,CA,95008,,,2023-05-19,10.0,10.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,95008,johna95008
1720554,SA11AI,C00401224,SA11AI_526451881,,,IND,,A,K,,,,3713 YUMA ST NW,,WASHINGTON,DC,20016,,,2023-04-25,5.0,5.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN VICTORY FUND (C00744946),,20016,ka20016
1720554,SA11AI,C00401224,SA11AI_531348463,,,IND,,A,LUIS,,,,106 STILLWOLD DRIVE,,WETHERSFIELD,CT,06109,,,2023-06-02,25.0,25.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,06109,luisa06109
1720554,SA11AI,C00401224,SA11AI_529907797,,,IND,,A,MARY,,,,15817 NW 173 RD STREET,,ALACHUA,FL,32615,,,2023-05-23,10.0,10.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN VICTORY FUND (C00744946),,32615,marya32615
1720554,SA11AI,C00401224,SA11AI_530180169,,,IND,,A,MICHAEL,,,,940 GRAND CONCOURSE,,NEW YORK,NY,10451,,,2023-05-25,10.0,10.0,Earmark,EONE,PRODUCER,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,10451,michaela10451
1720554,SA11AI,C00401224,SA11AI_527825689,,,IND,,A ADAMS,GARY,,,,3411 48TH STREET,,DES MOINES,IA,503103219,,,2023-05-04,25.0,25.0,Earmark,ISL EDUCATION LENDING,SCHOOL SERVICES LIAISON,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,50310,garya adams50310
1720554,SA11AI,C00401224,SA11AI_531049324,,,IND,,A ALLEY,JULIE,,,,5208 SW GENESEE ST,,"SEATTLE, WA",WA,98116,,,2023-05-31,25.0,25.0,Earmark,LIFELONG,IT MANAGER,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,98116,juliea alley98116
1720554,SA11AI,C00401224,SA11AI_526440924,,,IND,,A ANDREWS,SHARON,,,,1304 FOXX CT,,SUDLERSVILLE,MD,21668,,,2023-04-25,10.0,20.0,Earmark,NOT EMPLOYED,NOT EMPLOYED,,,,,,,,,,,,,,,,,,,Earmarked for BIDEN FOR PRESIDENT (C00703975),,21668,sharona andrews21668


In [23]:
%%sql
select min(contribution_date), max(contribution_date) from biden_harris_itemizations;

Running query in 'sqlite:///tmp.db'

min(contribution_date),max(contribution_date)
2023-04-25,2024-07-31


In [24]:

%%sql
select
  memo_text_description,
  count(*)
from biden_harris_itemizations
group by 1
order by 2 desc;

Running query in 'sqlite:///tmp.db'

memo_text_description,count(*)
Earmarked for BIDEN VICTORY FUND (C00744946),2389205
Earmarked for BIDEN FOR PRESIDENT (C00703975),2139849
Earmarked for HARRIS FOR PRESIDENT (C00703975),1766555
Earmarked for HARRIS VICTORY FUND (C00744946),1526606
Earmarked for BIDEN BALDWIN VICTORY FUND (C00849281),480
Earmarked for HARRIS BALDWIN VICTORY FUND (C00849281),29


In [25]:
%%sql
drop table if exists temp.contributor_stats;
create temp table contributor_stats as
select
  contributor_id,
  contributor_state,
  contributor_zip_code5,
  count(*) filter (where contribution_date < '2024-07-21') > 0 as biden_donor,
  count(*) filter (where contribution_date >= '2024-07-21') > 0 as harris_donor,
  sum(contribution_amount) as total_contribution_amount,
  sum(contribution_amount) filter (where contribution_date < '2024-07-21')  as biden_contribution_amount,
  sum(contribution_amount) filter (where contribution_date >= '2024-07-21') as harris_contribution_amount

from biden_harris_itemizations
group by 1;

Running query in 'sqlite:///tmp.db'

RuntimeError: (sqlite3.OperationalError) database table is locked
[SQL: drop table if exists temp.contributor_stats;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
%%sql
select
  contribution_date,
  count(distinct contributor_id)
from biden_harris_itemizations
where contribution_date between '2024-07-01' and '2024-07-20'
group by 1
order by 1;

Running query in 'sqlite:///tmp.db'

contribution_date,count(distinct contributor_id)
2024-07-01,48058
2024-07-02,46386
2024-07-03,40518
2024-07-04,32092
2024-07-05,34302
2024-07-06,34812
2024-07-07,22416
2024-07-08,29382
2024-07-09,25850
2024-07-10,23328


In [ ]:

%%sql
with biden_donors as materialized (
  select contributor_id
  from contributor_stats
  where biden_donor
)
select
  contribution_date,
  count(distinct contributor_id) filter (
    where contributor_id in biden_donors
  ) as biden_harris_donor,
  count(distinct contributor_id) filter (
    where contributor_id not in biden_donors
  ) as harris_only_supporter
from biden_harris_itemizations
where contribution_date between '2024-07-21' and '2024-08-01'
group by 1
order by 1;

Running query in 'sqlite:///tmp.db'

contribution_date,biden_harris_donor,harris_only_supporter
2024-07-21,208019,299486
2024-07-22,183605,414385
2024-07-23,83167,211140
2024-07-24,54875,122716
2024-07-25,44783,98375
2024-07-26,31739,68435
2024-07-27,38265,63320
2024-07-28,42495,68483
2024-07-29,66137,133476
2024-07-30,48328,83804


In [ ]:
%%sql
select
  sum(biden_donor) as num_biden_donors,
  sum(harris_donor) as num_harris_donors,
  --sum(harris_donor and not biden_donor) as num_harris_not_biden_donors,
  1.0 * sum(harris_donor and not biden_donor) / sum(harris_donor) as only_harris_ratios,
  sum(biden_contribution_amount) as biden_raised_total,
  sum(harris_contribution_amount) as harris_raised_total
from contributor_stats
where contributor_zip_code5 == '90601';

Running query in 'sqlite:///tmp.db'

num_biden_donors,num_harris_donors,only_harris_ratios,biden_raised_total,harris_raised_total
240,236,0.6864406779661016,24917.24,14852.5


In [ ]:
%config SqlMagic.displaylimit = None


displaylimit: Value None will be treated as 0 (no limit)

In [ ]:
%%sql
select count(*) from contributor_stats;

Running query in 'sqlite:///tmp.db'

count(*)
3511991


In [ ]:
%%sql
select * from contributor_stats limit 10;

Running query in 'sqlite:///tmp.db'

contributor_id,contributor_state,contributor_zip_code5,biden_donor,harris_donor,total_contribution_amount,biden_contribution_amount,harris_contribution_amount
adahlia romeroromero31707,GA,31707,0,1,2.0,None,2.0
davidmindak10011,NY,10011,1,0,15.0,15.0,None
halimachakouk60637,IL,60637,0,1,10.0,None,10.0
mindy leehappycampers92236,CA,92236,0,1,20.0,None,20.0
! mariaalfaro-brooks95492,CA,95492,0,1,500.0,None,500.0
!molliefair29412,SC,29412,1,0,100.0,100.0,None
# briansheehan80020,CO,80020,1,1,100.0,25.0,75.0
#2 douglas shimmerbrown23456,VA,23456,1,0,125.0,125.0,None
#balcurtis92651,CA,92651,1,0,200.0,200.0,None
#balwright17815,PA,17815,0,1,35.0,None,35.0


In [ ]:

%%sql
select
  contributor_state,
  count(),
  1.0 * sum(harris_donor and not biden_donor) / sum(harris_donor) as harris_not_biden_ratio
from contributor_stats
group by 1
order by 2 desc
limit 10;

Running query in 'sqlite:///tmp.db'

contributor_state,count(),harris_not_biden_ratio
CA,568607,0.6846144022703087
NY,253832,0.701840377323592
FL,210449,0.6764841980500239
TX,191440,0.7155082111238764
WA,141266,0.692919465678326
IL,134282,0.7132245969697328
PA,131852,0.676127126742036
MA,128121,0.693892934868509
VA,111363,0.7025438658304559
NC,107701,0.7095699352765124


In [ ]:
%%sql
select
  sum(contribution_amount),
  min(contribution_date),
  max(contribution_date)
  --sum(contribution_amount) filter (where contribution_amount < 200) as small_contribution_total,
  --sum(contribution_amount) filter (where contribution_amount >= 200) as large_contribution_total
from biden_harris_itemizations

Running query in 'sqlite:///tmp.db'

sum(contribution_amount),min(contribution_date),max(contribution_date)
434363067.05,2023-04-25,2024-07-31


In [ ]:
%%sql by_date <<
select
  contribution_date,
  'small' as contribution_type,
  sum(contribution_amount) as contribution_amount
from biden_harris_itemizations
where contribution_amount < 200
group by 1

union all

select
  contribution_date,
  'large' as contribution_type,
  sum(contribution_amount) as contribution_amount
from biden_harris_itemizations
where contribution_amount >= 200
group by 1

Running query in 'sqlite:///tmp.db'

In [ ]:
by_date.csv(filename="by-date-size2.csv")

CSV results at /Users/alex/projects/libfec/examples/latimes-harris-new/by-date-size2.csv

In [ ]:
%%sql
select contribution_date, sum("sum(contribution_amount)") over (order by contribution_date) from temp.x;

Running query in 'sqlite:///tmp.db'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(sqlite3.OperationalError) no such table: temp.x
[SQL: select contribution_date, sum("sum(contribution_amount)") over (order by contribution_date) from temp.x;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

If you need help solving this issue, send us a message: https://ploomber.io/community
